In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

embedding_model=OpenAIEmbeddings(disallowed_special=())

In [5]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")
collection_name="enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

vector_store = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = atlas_collection,
    index_name = index_name
)

retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 20}  # default "score_threshold": 0.75 
)

## RAG Pipeline - With HyDE

In [5]:
from langchain.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate
from langchain_groq import ChatGroq

def get_hypo_doc(query):
    template = """Imagine you are an expert writing a detailed explanation on the topic: '{query}'
    Your response should be comprehensive and include all key points that would be found in the top search result."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(template = template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
    messages = chat_prompt.format_prompt(query = query).to_messages()
    hyde_llm = ChatGroq(
        groq_api_key=os.getenv("GROD_CLOUD_API_KEY"),
        model_name='llama-3.1-70b-versatile'
    )
    response = hyde_llm.invoke(messages)
    hypo_doc = response.content
    return hypo_doc

In [14]:
def call_openai(question):
   
   query_optimized = get_hypo_doc(question['question'])

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {query_optimized}"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def remove_rep(docs):
      docs_content = []
      unique_docs = []
      for doc in docs:
            if doc.page_content not in docs_content:
                  docs_content.append(doc.page_content)
                  unique_docs.append(doc)
      return unique_docs
   
   def format_docs(docs):
      contexts = remove_rep(docs)
      return "\n\n".join(doc.page_content for doc in contexts)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "query_optimized": query_optimized | RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   answer = rag_chain.invoke(question)
   retrieved_docs = remove_rep(retriever.invoke(query_optimized))

   return{
      'answer': answer,
      'contexts': retrieved_docs
      }

### Test the RAG pipeline

In [15]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai(question)
print(answer['answer'])

The article titled "The Power of Supportive Culture: Unlocking Employee Engagement and Aligning with Tech Innovators Inc.'s Approach" emphasizes the significance of a supportive culture in enhancing employee engagement and organizational success. At Tech Innovators Inc., a supportive culture is defined as an environment that values, respects, and empowers employees, characterized by open communication, trust, empathy, and community. 

The impact of such a culture on employee engagement is profound. Employees who feel supported tend to be more motivated, productive, and resilient. They also develop a sense of belonging and are more likely to remain with the organization, which helps reduce turnover rates.

Tech Innovators Inc. aligns its approach to employment relations and engagement with the principles of open communication, employee empowerment, prioritization of well-being, commitment to diversity, equity, and inclusion, and continuous learning and development. 

To implement a supp

In [16]:
answer['contexts']

[Document(metadata={'_id': '66d8163d0533e009aa2d4b0f', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture'),
 Document(metadata={'_id': '66d8163d0533e009aa2d4b1c', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='concept that requires a holistic approach. At Tech Innovators Inc., we are committed to fostering an engaging work environment through open communication, recognition, and support. By understanding'),
 Document(metadata={'_id': '66d8163d0533e009aa2d4b10', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='supportive culture where employees feel valued and respect

# RAG pipeline evaluation

## Test data set prep

In [9]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [12]:
def extract_page_content(documents):
    return [doc.page_content for doc in documents]

In [11]:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test-dataset/",file_name)
data_to_test = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [ ]:
# Generate all the answers for the questions in the dataset
answers = []
contexts = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    contexts.append(answer['contexts'])
    answers.append(answer['answer'])

In [31]:
# update the dataset with answers
data_to_test['answer'] = answers
data_to_test['contexts'] = contexts

In [19]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts'] = data_to_test['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [28]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer'])
contexts = list(data_to_test['contexts'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_hyde = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_hyde = Dataset.from_dict(data_hyde)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [30]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset_hyde,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'answer_relevancy': 0.8666, 'faithfulness': 0.5302, 'context_recall': 0.5375, 'context_precision': 0.6542}

In [31]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The role of the Senior Director responsible fo...,[the Senior Director responsible for Analytics...,The role of the Senior Director responsible fo...,0.962325,0.736842,0.272727,0.8875
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[to identify strengths and areas for improveme...,Identifying and addressing growth areas in sel...,0.990183,0.857143,0.333333,0.0000
2,What forms of unethical behavior are strictly ...,"In the recruitment process of Inc., any form o...",[Inc. upholds the highest ethical standards in...,Favoritism or nepotism,0.979409,0.500000,1.000000,0.7500
3,What is the significance of emotional and aest...,Emotional and aesthetic labor are significant ...,[LabourEmotional and aesthetic labor involves ...,Emotional and aesthetic labor in the workplace...,0.974147,0.833333,1.000000,1.0000
4,What is the purpose of the orientation session...,I don't have the specific details about the or...,[see you thrive at Tech Innovators Inc. Welcom...,The purpose of the orientation session at Tech...,0.000000,0.375000,0.000000,0.0000
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., employees have access...",[and identify areas for improvement.5.3 Report...,Employees can report violations of labor laws ...,0.962563,0.166667,0.250000,1.0000
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.960525,0.636364,1.000000,1.0000
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[IntroductionThis guide provides a step-by-ste...,To extract data from Confluence and create a R...,0.989648,0.416667,0.181818,1.0000
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,[IntroductionTech Innovators Inc. is committed...,Tech Innovators Inc. promotes employee engagem...,0.980843,0.250000,0.800000,0.2500


In [32]:
folder = "data-test/eval-result/eval_result_test_dataset_it_with_hyde.csv"
json_file_path = os.path.join(from_root(), folder)
df.to_csv(json_file_path, index=False)